In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load all dependencies
import sys
import os
import torch
from einops import rearrange

sys.path.append('../')

from src.dataset.jukebox_dataset import JukeboxDataset
from src.model.jukebox_diffusion import JukeboxDiffusion
from src.module.diffusion_attn_unet_1d import DiffusionAttnUnet1D

from IPython.display import Audio

In [ ]:
def play_audio(audio: torch.Tensor):
    for a in torch.clamp(audio, -1, 1).cpu().numpy():
        display(Audio(a.flatten(), rate=44100))

In [ ]:
ckpt_path = '../logs/train/runs/2022-12-09_19-50-48/checkpoints/last.ckpt'
ckpt = torch.load(ckpt_path)

In [ ]:

seqmodel = DiffusionAttnUnet1D(
        io_channels=64,
        n_attn_layers=6,
        channel_sizes=[128, 128, 128, 128,  256, 256, 256, 256,  512, 512, 512]
    )

# remove all keys from state_dict that start with 'jukebox'
state_dict = {k: v for k, v in ckpt['state_dict'].items() if not k.startswith('jukebox_vqvae')}

model = JukeboxDiffusion(model=seqmodel)
model.load_state_dict(state_dict=state_dict)
model.hparams.jukebox_embedding_lvl = 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f'Using device: {device}')

In [ ]:
model.prepare_data()

In [ ]:
# check that decoding works
dataset = JukeboxDataset(root_dir=os.environ['MAESTRO_DATASET_DIR'], split='train', lvl=2, sequence_len=4096)
test_audio = model.preprocess(dataset[0].unsqueeze(0).to(device))

In [ ]:
with torch.no_grad():
    play_audio(model.decode(test_audio))

In [ ]:
with torch.no_grad():
    embeddings = model.generate_unconditionally(
                batch_size=4,
                seq_len=4096,
                num_inference_steps=200,
                seed=21,
            )
    audio = model.decode(embeddings)

In [ ]:
play_audio(audio)